# Generación de Música con Cadenas de Markov

### Contenido:
- **Parte A**: Método tradicional (Markov sobre notas)
- **Parte B**: Método de Xu (2023) - Markov para acordes + Interpolación para melodía (CORREGIDO)
- **Parte C**: Evaluación

In [ ]:
import numpy as np
from collections import defaultdict, Counter
from music21 import converter, note, chord, stream, instrument, pitch, key
from typing import List, Tuple, Dict, Any
from scipy.stats import entropy
from scipy.interpolate import PchipInterpolator  # Monotonic, no oscillation!
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

def get_project_root() -> Path:
    current = Path.cwd()
    if (current / 'midis').exists():
        return current
    for parent in current.parents:
        if (parent / 'midis').exists():
            return parent
    ubicaciones_conocidas = [
        Path.home() / 'Documents/universidad/2025-II/cadenas/proyecto/esnupi_de_markov',
    ]
    for ubicacion in ubicaciones_conocidas:
        if ubicacion.exists() and (ubicacion / 'midis').exists():
            return ubicacion
    raise FileNotFoundError("No se encontró el directorio del proyecto.")

PROJECT_ROOT = get_project_root()
print(f"Directorio del proyecto: {PROJECT_ROOT}")

---
# PARTE A: Cadena de Markov
---

In [ ]:
class CadenaDeMarkov:
    def __init__(self, orden: int = 2):
        self.orden = orden
        self.transiciones: Dict[tuple, Counter] = defaultdict(Counter)
        self.secuencias_iniciales: List[tuple] = []
        self.estados: set = set()
    
    def entrenar(self, secuencia: List[Any]) -> None:
        if len(secuencia) < self.orden + 1:
            return
        self.secuencias_iniciales.append(tuple(secuencia[:self.orden]))
        self.estados.update(secuencia)
        for i in range(len(secuencia) - self.orden):
            contexto = tuple(secuencia[i:i + self.orden])
            siguiente = secuencia[i + self.orden]
            self.transiciones[contexto][siguiente] += 1
    
    def obtener_probabilidades(self, contexto: tuple) -> Dict[Any, float]:
        conteos = self.transiciones[contexto]
        total = sum(conteos.values())
        if total == 0:
            return {}
        return {estado: conteo / total for estado, conteo in conteos.items()}
    
    def generar_siguiente(self, contexto: List[Any], temperatura: float = 1.0) -> Any:
        ctx = tuple(contexto[-self.orden:]) if len(contexto) >= self.orden else tuple(contexto)
        conteos = self.transiciones.get(ctx, {})
        if not conteos:
            return list(self.estados)[np.random.randint(len(self.estados))] if self.estados else None
        estados = list(conteos.keys())
        probs = np.array([conteos[e] for e in estados], dtype=float)
        probs /= probs.sum()
        if temperatura != 1.0 and len(probs) > 1:
            probs = np.power(probs, 1.0 / temperatura)
            probs /= probs.sum()
        return estados[np.random.choice(len(estados), p=probs)]
    
    def generar_secuencia(self, longitud: int, temperatura: float = 1.0) -> List[Any]:
        if not self.secuencias_iniciales:
            return []
        idx = np.random.randint(len(self.secuencias_iniciales))
        secuencia = list(self.secuencias_iniciales[idx])
        while len(secuencia) < longitud:
            siguiente = self.generar_siguiente(secuencia, temperatura)
            if siguiente is None:
                break
            secuencia.append(siguiente)
        return secuencia[:longitud]

In [ ]:
def extraer_eventos(ruta_midi: str, umbral_acorde: float = 0.05) -> List[Dict]:
    partitura = converter.parse(ruta_midi)
    eventos = []
    for parte in partitura.parts:
        notas_con_tiempo = []
        for elem in parte.flatten().notesAndRests:
            if isinstance(elem, note.Note):
                notas_con_tiempo.append({
                    'offset': float(elem.offset),
                    'notas': [elem.pitch.nameWithOctave],
                    'duracion': float(elem.duration.quarterLength),
                    'midi': elem.pitch.midi
                })
            elif isinstance(elem, chord.Chord):
                notas_con_tiempo.append({
                    'offset': float(elem.offset),
                    'notas': sorted([p.nameWithOctave for p in elem.pitches]),
                    'duracion': float(elem.duration.quarterLength),
                    'midi': [p.midi for p in elem.pitches]
                })
        agrupados = agrupar_por_tiempo(notas_con_tiempo, umbral_acorde)
        eventos.extend(agrupados)
    eventos.sort(key=lambda x: x['offset'])
    return eventos

def agrupar_por_tiempo(notas: List[Dict], umbral: float) -> List[Dict]:
    if not notas:
        return []
    notas = sorted(notas, key=lambda x: x['offset'])
    grupos = []
    actual = {'offset': notas[0]['offset'], 'notas': list(notas[0]['notas']), 'duracion': notas[0]['duracion']}
    for n in notas[1:]:
        if abs(n['offset'] - actual['offset']) <= umbral:
            actual['notas'].extend(n['notas'])
            actual['duracion'] = max(actual['duracion'], n['duracion'])
        else:
            grupos.append(actual)
            actual = {'offset': n['offset'], 'notas': list(n['notas']), 'duracion': n['duracion']}
    grupos.append(actual)
    for g in grupos:
        g['nota'] = '.'.join(sorted(set(g['notas']))) if len(g['notas']) > 1 else g['notas'][0]
    return grupos

In [ ]:
ARCHIVO_MIDI = PROJECT_ROOT / 'midis/ZeldaFantasy_1_.mid'
ORDEN = 2

print(f"Cargando: {ARCHIVO_MIDI}")
eventos = extraer_eventos(str(ARCHIVO_MIDI))
print(f"Eventos extraídos: {len(eventos)}")
print(f"Notas únicas: {len(set(e['nota'] for e in eventos))}")

secuencia_conjunta = [(e['nota'], e['duracion']) for e in eventos]
cadena = CadenaDeMarkov(orden=ORDEN)
cadena.entrenar(secuencia_conjunta)
print(f"Cadena entrenada: {len(cadena.transiciones)} contextos")

In [ ]:
def guardar_midi(melodia: List[Tuple], ruta_salida: str, con_acordes: bool = False):
    partitura = stream.Score()
    parte_melodia = stream.Part()
    parte_melodia.insert(0, instrument.Piano())
    parte_acordes = stream.Part()
    parte_acordes.insert(0, instrument.Piano())
    
    offset_actual = 0.0
    for item in melodia:
        if len(item) == 2:
            nota_str, duracion = item
            acorde_notas = None
        else:
            nota_str, duracion, acorde_notas = item
        
        duracion = max(0.125, float(duracion))
        
        if '.' in nota_str:
            notas = nota_str.split('.')
            elem = chord.Chord(notas)
        else:
            elem = note.Note(nota_str)
        elem.duration.quarterLength = duracion
        elem.volume.velocity = 90
        parte_melodia.insert(offset_actual, elem)
        
        if acorde_notas and con_acordes:
            try:
                acorde_elem = chord.Chord(acorde_notas)
                acorde_elem.duration.quarterLength = duracion
                acorde_elem.volume.velocity = 50
                parte_acordes.insert(offset_actual, acorde_elem)
            except:
                pass
        
        offset_actual += duracion
    
    partitura.insert(0, parte_melodia)
    if con_acordes:
        partitura.insert(0, parte_acordes)
    partitura.write('midi', fp=ruta_salida)
    print(f"Guardado: {ruta_salida}")

In [ ]:
print("=" * 50)
print("GENERANDO MELODÍA (Método Tradicional)")
print("=" * 50)

melodia_tradicional = cadena.generar_secuencia(100, temperatura=1.2)
print(f"Generados: {len(melodia_tradicional)} eventos")

output_dir = PROJECT_ROOT / 'generated_music'
output_dir.mkdir(parents=True, exist_ok=True)
nombre_base = ARCHIVO_MIDI.stem
guardar_midi(melodia_tradicional, str(output_dir / f'{nombre_base}_tradicional.mid'))

---
# PARTE B: Método Xu CORREGIDO
---

### Problemas del Lagrange original:
1. ❌ Lagrange oscila salvajemente (fenómeno de Runge)
2. ❌ Los valores interpolados no tienen sentido musical
3. ❌ No respeta la estructura armónica

### Solución implementada:
1. ✅ **PCHIP** (Piecewise Cubic Hermite) - interpolación monótona SIN oscilaciones
2. ✅ **Contorno melódico** basado en grados de escala (no MIDI crudo)
3. ✅ **Markov sobre intervalos** aprendidos del original
4. ✅ **Notas de paso y bordaduras** para conectar acordes

In [ ]:
class GeneradorXuCorregido:
    """
    Método de Xu CORREGIDO:
    - Usa PCHIP en lugar de Lagrange (sin oscilaciones)
    - Aprende patrones melódicos del MIDI original
    - Genera melodía coherente + acordes
    """
    
    def __init__(self, ruta_midi: str):
        self.ruta_midi = ruta_midi
        self.partitura = converter.parse(ruta_midi)
        self.tonalidad = self._detectar_tonalidad()
        self.escala_midi = self._construir_escala_midi()  # MIDI values de la escala
        self.cadena_acordes = CadenaDeMarkov(orden=1)
        self.cadena_grados = CadenaDeMarkov(orden=2)  # Grados de escala, no MIDI
        self.ritmos_originales = []
        self.rango_octava = (3, 5)  # Octavas válidas
        
        self._definir_acordes()
        
    def _detectar_tonalidad(self) -> key.Key:
        try:
            k = self.partitura.analyze('key')
            print(f"Tonalidad detectada: {k}")
            return k
        except:
            print("Usando E menor por defecto")
            return key.Key('e', 'minor')
    
    def _construir_escala_midi(self) -> List[int]:
        """Construye lista de valores MIDI para todas las notas de la escala en varias octavas."""
        escala_notas = [p.name for p in self.tonalidad.pitches[:7]]
        midi_vals = []
        for octava in range(2, 7):
            for nota in escala_notas:
                try:
                    midi_vals.append(pitch.Pitch(f"{nota}{octava}").midi)
                except:
                    pass
        return sorted(set(midi_vals))
    
    def _midi_a_grado(self, midi_val: int) -> int:
        """Convierte MIDI a grado de escala (0-6)."""
        escala_notas = [p.name for p in self.tonalidad.pitches[:7]]
        try:
            p = pitch.Pitch(midi=midi_val)
            nombre = p.name
            if nombre in escala_notas:
                return escala_notas.index(nombre)
            # Buscar el más cercano
            for i, n in enumerate(escala_notas):
                if n[0] == nombre[0]:  # Misma letra
                    return i
        except:
            pass
        return 0
    
    def _grado_a_midi(self, grado: int, octava: int) -> int:
        """Convierte grado de escala a MIDI."""
        escala_notas = [p.name for p in self.tonalidad.pitches[:7]]
        grado = grado % 7
        try:
            return pitch.Pitch(f"{escala_notas[grado]}{octava}").midi
        except:
            return 60
    
    def _definir_acordes(self):
        modo = self.tonalidad.mode
        escala = [p.name for p in self.tonalidad.pitches[:7]]
        
        self.acordes = {}
        self.nombres = {}
        nombres_maj = ['I', 'ii', 'iii', 'IV', 'V', 'vi', 'vii°']
        nombres_min = ['i', 'ii°', 'III', 'iv', 'v', 'VI', 'VII']
        
        for i in range(7):
            raiz = escala[i]
            tercera = escala[(i + 2) % 7]
            quinta = escala[(i + 4) % 7]
            self.acordes[i + 1] = [raiz, tercera, quinta]
            self.nombres[i + 1] = nombres_maj[i] if modo == 'major' else nombres_min[i]
        
        print(f"Escala: {escala}")
        print(f"Acordes: {self.nombres}")
    
    def _extraer_del_midi(self):
        """Extrae progresiones, grados melódicos y ritmos."""
        notas_por_compas = defaultdict(list)
        grados_melodicos = []  # Secuencia de grados (0-6)
        
        for parte in self.partitura.parts:
            for elem in parte.flatten().notesAndRests:
                if isinstance(elem, note.Note):
                    compas = int(elem.offset // 4)
                    notas_por_compas[compas].append(elem.pitch.name)
                    grados_melodicos.append(self._midi_a_grado(elem.pitch.midi))
                    self.ritmos_originales.append(float(elem.duration.quarterLength))
                elif isinstance(elem, chord.Chord):
                    compas = int(elem.offset // 4)
                    for p in elem.pitches:
                        notas_por_compas[compas].append(p.name)
                    grados_melodicos.append(self._midi_a_grado(elem.pitches[-1].midi))
                    self.ritmos_originales.append(float(elem.duration.quarterLength))
        
        # Entrenar Markov sobre GRADOS (no MIDI)
        if grados_melodicos:
            self.cadena_grados.entrenar(grados_melodicos)
            print(f"Grados melódicos aprendidos: {len(self.cadena_grados.transiciones)} contextos")
        
        # Inferir acordes
        progresion = []
        for compas in sorted(notas_por_compas.keys()):
            notas = notas_por_compas[compas]
            progresion.append(self._inferir_acorde(notas))
        
        if not self.ritmos_originales:
            self.ritmos_originales = [0.5, 1.0, 0.5, 1.0, 0.25]
        
        return progresion
    
    def _inferir_acorde(self, notas: List[str]) -> int:
        if not notas:
            return 1
        contador = Counter(notas)
        mejor, mejor_score = 1, 0
        for grado, notas_ac in self.acordes.items():
            score = sum(contador.get(n, 0) for n in notas_ac)
            if score > mejor_score:
                mejor_score = score
                mejor = grado
        return mejor
    
    def entrenar(self):
        prog = self._extraer_del_midi()
        if prog:
            self.cadena_acordes.entrenar(prog)
            print(f"Acordes del MIDI: {len(prog)}")
        
        # Progresiones comunes
        for p in [[1,5,6,4], [1,4,5,1], [6,4,1,5], [1,6,4,5], [1,5,4,1]]:
            self.cadena_acordes.entrenar(p)
        
        print(f"Total transiciones de acordes: {len(self.cadena_acordes.transiciones)}")
    
    def generar_acordes(self, n: int) -> List[int]:
        prog = [1]
        while len(prog) < n:
            sig = self.cadena_acordes.generar_siguiente(prog)
            prog.append(sig if sig else np.random.randint(1, 8))
        prog[-1] = 1
        return prog
    
    def _crear_contorno_pchip(self, acordes: List[int], octava: int = 4) -> callable:
        """
        Crea contorno melódico usando PCHIP (no oscila como Lagrange).
        Los puntos de control son la RAÍZ de cada acorde.
        """
        x_puntos = []
        y_puntos = []
        
        nota_anterior = None
        for i, ac in enumerate(acordes):
            raiz = self.acordes[ac][0]  # Raíz del acorde
            
            # Elegir octava para voice leading suave
            if nota_anterior is None:
                midi_val = pitch.Pitch(f"{raiz}{octava}").midi
            else:
                # Buscar la octava más cercana
                candidatos = [pitch.Pitch(f"{raiz}{o}").midi for o in range(3, 6)]
                midi_val = min(candidatos, key=lambda x: abs(x - nota_anterior))
            
            x_puntos.append(i)
            y_puntos.append(midi_val)
            nota_anterior = midi_val
        
        # PCHIP: Piecewise Cubic Hermite Interpolating Polynomial
        # NO oscila como Lagrange, es monótona entre puntos
        if len(x_puntos) >= 2:
            return PchipInterpolator(x_puntos, y_puntos)
        else:
            return lambda x: y_puntos[0] if y_puntos else 60
    
    def _cuantizar_a_escala(self, midi_val: float) -> str:
        """Cuantiza valor MIDI a la nota más cercana EN LA ESCALA."""
        midi_val = int(round(midi_val))
        # Buscar la nota de escala más cercana
        if self.escala_midi:
            cercana = min(self.escala_midi, key=lambda x: abs(x - midi_val))
            return pitch.Pitch(midi=cercana).nameWithOctave
        return pitch.Pitch(midi=midi_val).nameWithOctave
    
    def generar_melodia(self, acordes: List[int], notas_por_compas: int = 4):
        """
        Genera melodía usando:
        1. PCHIP para el contorno general (suave, sin oscilaciones)
        2. Markov sobre grados para variación melódica
        3. Notas del acorde como anclas
        """
        octava = 4
        contorno = self._crear_contorno_pchip(acordes, octava)
        
        melodia = []
        grado_actual = 0  # Empezar en tónica
        contexto_grados = [0, 0]
        
        for i, ac in enumerate(acordes):
            notas_acorde = self.acordes[ac]
            # Acorde de acompañamiento (una octava abajo)
            acorde_acomp = [f"{n}{octava - 1}" for n in notas_acorde]
            
            # Generar notas para este compás
            num_notas = np.random.randint(3, 6)
            
            for j in range(num_notas):
                t = i + j / num_notas  # Posición en el tiempo
                
                if j == 0:
                    # Primera nota: usar contorno PCHIP (raíz del acorde)
                    midi_objetivo = float(contorno(t))
                    nota_str = self._cuantizar_a_escala(midi_objetivo)
                elif j == num_notas - 1:
                    # Última nota: preparar siguiente acorde
                    midi_objetivo = float(contorno(t))
                    nota_str = self._cuantizar_a_escala(midi_objetivo)
                else:
                    # Notas intermedias: usar Markov sobre grados
                    if self.cadena_grados.transiciones:
                        nuevo_grado = self.cadena_grados.generar_siguiente(contexto_grados)
                        if nuevo_grado is not None:
                            grado_actual = nuevo_grado
                    else:
                        # Fallback: nota del acorde
                        grado_actual = self._midi_a_grado(
                            pitch.Pitch(f"{np.random.choice(notas_acorde)}{octava}").midi
                        )
                    
                    midi_objetivo = self._grado_a_midi(grado_actual, octava)
                    nota_str = self._cuantizar_a_escala(midi_objetivo)
                    contexto_grados = contexto_grados[1:] + [grado_actual]
                
                # Ritmo del original
                dur = np.random.choice(self.ritmos_originales)
                dur = max(0.25, min(dur, 2.0))
                
                melodia.append((nota_str, dur, acorde_acomp))
        
        return melodia
    
    def generar_pieza(self, n_compases: int = 16):
        acordes = self.generar_acordes(n_compases)
        melodia = self.generar_melodia(acordes)
        return acordes, melodia
    
    def acordes_nombres(self, prog):
        return [self.nombres.get(a, '?') for a in prog]

In [ ]:
print("=" * 50)
print("GENERANDO MELODÍA (Método Xu CORREGIDO - PCHIP)")
print("=" * 50)

gen_xu = GeneradorXuCorregido(str(ARCHIVO_MIDI))
gen_xu.entrenar()

acordes_xu, melodia_xu = gen_xu.generar_pieza(16)

print(f"\nProgresión: {' - '.join(gen_xu.acordes_nombres(acordes_xu))}")
print(f"Notas generadas: {len(melodia_xu)}")

# Mostrar primeras notas
print(f"\nPrimeras 10 notas: {[n for n, d, a in melodia_xu[:10]]}")

guardar_midi(melodia_xu, str(output_dir / f'{nombre_base}_xu_lagrange.mid'), con_acordes=True)

---
# PARTE C: Evaluación
---

In [ ]:
def calcular_cross_entropy(cadena, secuencia):
    if len(secuencia) < cadena.orden + 1:
        return float('inf')
    log_probs = []
    for i in range(cadena.orden, len(secuencia)):
        contexto = tuple(secuencia[i - cadena.orden:i])
        siguiente = secuencia[i]
        probs = cadena.obtener_probabilidades(contexto)
        if siguiente in probs and probs[siguiente] > 0:
            log_probs.append(np.log2(probs[siguiente]))
        else:
            log_probs.append(np.log2(1e-10))
    return -np.mean(log_probs) if log_probs else float('inf')

def jensen_shannon(lista1, lista2):
    f1, f2 = Counter(lista1), Counter(lista2)
    vocab = set(f1.keys()) | set(f2.keys())
    eps = 1e-10
    p = np.array([f1.get(v, 0) + eps for v in vocab])
    q = np.array([f2.get(v, 0) + eps for v in vocab])
    p, q = p/p.sum(), q/q.sum()
    m = 0.5 * (p + q)
    return float(0.5 * entropy(p, m) + 0.5 * entropy(q, m))

def evaluar_melodia(nombre: str, melodia, cadena, eventos_orig):
    """Evalúa una melodía con todas las métricas."""
    # Manejar tanto tuplas de 2 como de 3 elementos
    if len(melodia[0]) == 3:
        melodia_simple = [(n, d) for n, d, _ in melodia]
    else:
        melodia_simple = melodia
    
    notas_gen = [n for n, _ in melodia_simple]
    notas_orig = [e['nota'] for e in eventos_orig]
    
    ce = calcular_cross_entropy(cadena, melodia_simple)
    js = jensen_shannon(notas_orig, notas_gen)
    
    print(f"\n{'='*50}")
    print(f"📊 {nombre}")
    print(f"{'='*50}")
    print(f"   Cross-Entropy: {ce:.2f} bits")
    print(f"   JS Divergence: {js:.4f}")
    return {'ce': ce, 'js': js}

In [ ]:
print("\n" + "="*60)
print("   COMPARACIÓN")
print("="*60)

evaluar("Tradicional", melodia_tradicional, cadena, eventos)
evaluar("Xu Corregido (PCHIP)", melodia_xu, cadena, eventos)

---
## ¿Por qué Lagrange no funciona para melodías?

### El problema matemático:
```
Lagrange con n puntos = polinomio de grado n-1
→ Oscila ENTRE los puntos de control
→ "Fenómeno de Runge": oscilaciones extremas en los bordes
```

### Ejemplo:
Si tienes acordes C → G → Am → F, Lagrange podría generar:
- C4 (60) → **B7 (95)** → **D2 (38)** → G4 (67) → ...

Esos saltos de 35 semitonos NO son música.

### Solución: PCHIP
- **P**iecewise **C**ubic **H**ermite **I**nterpolating **P**olynomial
- Garantiza que la curva sea **monótona** entre puntos
- NO hay oscilaciones salvajes
- Resultado: contorno melódico suave y musical